# calculate pentadal running mean
- Period 1. 1965-1986 OSD (1967-1986)
- Period 2. 1987-2009 OSD+CTD (1987-2020)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
import xarray as xr
import pandas as pd
import os

In [ ]:
# full depth monthly climatology
dclim=xr.open_dataset('o2_clim_OI1x1_osdctd_pass2.nc')
Nlev=47
z=dclim.depth[:Nlev].to_numpy()
Nz=Nlev
diro='intermed_files/'

In [ ]:
o2clim=dclim.o2.to_numpy()

In [ ]:
year=range(1965,2022)
fn0='o2_1x1bin_osd_'
fn1='o2_1x1bin_ctd_'
wn='o2_1x1bin_osdctd_'
x=dclim.lon
y=dclim.lat
for yr in year:
    print(str(yr))
    ds=xr.open_dataset(diro+fn0+str(yr)+'.nc')
    a1=ds.o2.to_numpy()-o2clim
    a1=np.nanmean(a1,axis=0)
    a1[np.isnan(a1)]=0
    # aggregate into annual bin
    n1=ds.sample_count.sum('mon').to_numpy()
    # if after 1987, include CTD data
    a=np.zeros((1,Nlev,180,360))
    n=np.zeros((1,Nlev,180,360))
    if (yr < 1987):
        a[0,:,:,:]=a1
        n[0,:,:,:]=n1
    else:
        ds=xr.open_dataset(diro+fn1+str(yr)+'.nc')
        a2=ds.o2.to_numpy()-o2clim
        a2=np.nanmean(a2,axis=0)
        a2[np.isnan(a2)]=0
        # aggregate into annual bin
        n2=ds.sample_count.sum('mon').to_numpy()
        # then combine with bottle data
        a[0,:,:,:]=a1*n1+a2*n2
        n[0,:,:,:]=n1+n2
    a=a/n
    # write out the output files
    wn0=wn+str(yr)+'.nc'
    da=xr.DataArray(data=a,name='o2anom',dims=['year','depth','lat','lon'],\
                   coords={'year':[yr],'depth':z,'lat':y,'lon':x})
    dn=xr.DataArray(data=n,name='sample_count',dims=['year','depth','lat','lon'],\
                   coords={'year':[yr],'depth':z,'lat':y,'lon':x})
    ds1=da.to_dataset()
    ds1['sample_count']=dn
    ds1.to_netcdf(diro+wn0)

In [ ]:
# calculate 5 year running mean
year=range(1967,2020)
fn='o2_1x1bin_osdctd_'
for yr in year:
    print(yr)
    wn=diro+'o2_5yrm_'+str(yr)+'.nc'
    d=np.empty((5,Nlev,180,360))
    d2=np.empty((5,Nlev,180,360))
    d[0,:,:,:]=xr.open_dataset(diro+fn+str(yr-2)+'.nc').o2anom.to_numpy()
    d[1,:,:,:]=xr.open_dataset(diro+fn+str(yr-1)+'.nc').o2anom.to_numpy()
    d[2,:,:,:]=xr.open_dataset(diro+fn+str(yr)+'.nc').o2anom.to_numpy()
    d[3,:,:,:]=xr.open_dataset(diro+fn+str(yr+1)+'.nc').o2anom.to_numpy()
    d[4,:,:,:]=xr.open_dataset(diro+fn+str(yr+2)+'.nc').o2anom.to_numpy()
    #
    d2[0,:,:,:]=xr.open_dataset(diro+fn+str(yr-2)+'.nc').sample_count.to_numpy()
    d2[1,:,:,:]=xr.open_dataset(diro+fn+str(yr-1)+'.nc').sample_count.to_numpy()
    d2[2,:,:,:]=xr.open_dataset(diro+fn+str(yr)+'.nc').sample_count.to_numpy()
    d2[3,:,:,:]=xr.open_dataset(diro+fn+str(yr+1)+'.nc').sample_count.to_numpy()
    d2[4,:,:,:]=xr.open_dataset(diro+fn+str(yr+2)+'.nc').sample_count.to_numpy()
    #
    d0=np.nanmean(d,axis=0)
    d1=np.where(d0==0,np.nan,d0)
    d20=np.nansum(d2,axis=0)
    ds=xr.open_dataset(diro+fn+str(yr)+'.nc')
    #
    da=xr.DataArray(data=d1,name='o2anom',dims=['depth','lat','lon'],\
                    coords={'depth':ds.depth,'lat':ds.lat,'lon':ds.lon})
    da2=xr.DataArray(data=d20,name='sample_count',dims=['depth','lat','lon'],\
                    coords={'depth':ds.depth,'lat':ds.lat,'lon':ds.lon})
    ds1=da.to_dataset()
    ds1['sample_count']=da2
    os.system('rm '+wn)
    ds1.to_netcdf(wn)

In [ ]:
ds1=xr.open_mfdataset(diro+'o2_5yrm_1987.nc')
plt.pcolormesh(ds1.o2anom[0,:,:],vmin=-20,vmax=20,cmap='RdBu_r')
plt.colorbar()

In [ ]:
os.system('ls intermed_files/o2_5yrm*.nc > fn.txt')